In [1]:
from casadi import *
from casadi.tools import *
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
import matplotlib
import pickle
import os

# Functions to estimate the optimal trajectory

In [2]:
def penalty(x, k, mu, amp):
    """Generate the spatial cost (approximation of the Heaviside function)
    Args:
        x: scalar, list or array 
        k: scalar, steepness of the spatial penalty function 
        mu: scalar, beam location
        amp: scalar, height of the spatial penalty function
    Returns:
        p: scalar, list or array, value of the spatial cost for x 
    """  
    return(amp/(1+np.exp(1+k*(x-mu))))

def optimal_trajectory(cost, a, b, xb, xi, xf, tf, vtapis, tau_res, mass, ampxpenalty, kxpenalty, nk):
    """Compute the optimal trajectory
    Args:
        cost: string, cost type (defines the effort and spacial costs)  
        a: scalar, effort sensitivity 
        b: scalar, spacial sensitivity
        xb: scalar, beam location
        xi: scalar, initial position
        xf: scalar, final position
        tf: scalar, final time (entrance time)
        vtapis: scalar, treadmill speed
        tau_res: scalar, resistive force, tau
        mass: scalar, rat's weight (mass)
        ampxpenalty: scalar, height of the spatial penalty function
        kxpenalty: scalar, steepness of the spatial penalty function 
        nk: scalar, number of collocation points
    Returns:
        (x0_opt,x1_opt,tgrid,u_opt,tgrid_u,float(res["f"])): lists; optimal positions, optimal speeds
        time for postion and speed), optimal control, time for optimal control, final total cost. 

    """
    
    Ltread=0.9
    mu=0.1    # Control discretization
    kT=mu/1
    
    # Declare variables
    t  = SX.sym("t")    # time
    u  = SX.sym("u")    # control

    states = struct_symSX([
            entry('x',shape=2),    #  states
            entry('L')             #  helper state: Langrange integrand
         ])

    # Create a structure for the right hand side
    rhs = struct_SX(states)
    x = states['x']
    rhs["x"] = vertcat(x[1],u)    
    rhs["x"] = vertcat(x[1],u - fabs(x[1]-vtapis)/tau_res)
        
    
    if cost=='speed_quadratic':
        
        rhs["L"] = a*mass*(x[1]-vtapis)**2 + b*(x[0]-Ltread)**2
    
    elif cost=='force_quadratic':   
        rhs["L"] = a*(mass*u)**2 + b*(x[0]-Ltread)**2  

    elif cost=='speed_heaviside':
        
        rhs["L"] = a*mass*(x[1]-vtapis)**2 + b*penalty(x=x[0],k=kxpenalty,mu=xb,amp=ampxpenalty)
    
    elif cost=='force_heaviside':   
        rhs["L"] = a*(mass*u)**2 + b*penalty(x=x[0],k=kxpenalty,mu=xb,amp=ampxpenalty)

    # ODE right hand side function
    f = Function('f', [t,states,u],[rhs])

    # Objective function (meyer term)
    m = Function('m', [t,states,u],[states["L"]])

    # Control bounds
    u_min = -7.5
    u_max = 7.5
    u_init = 0.0

    u_lb = np.array([u_min])
    u_ub = np.array([u_max])
    u_init = np.array([u_init])

    # State bounds and initial guess
    x_min =  [0.0,   vtapis-0.7, -inf]
    x_max =  [Ltread,vtapis,  inf]
    xi_min = [xi,  vtapis-0.7,  0.0]
    xi_max = [xi,  vtapis,  inf]
    xf_min = [xf,  vtapis-0.7, -inf]
    xf_max = [xf,  vtapis,  inf]
    x_init = [ 0.0,  0.0,  0.0]

    # Dimensions
    nx = 3
    nu = 1

    # Choose collocation points
    tau_root = [0] + collocation_points(3,"radau")

    # Degree of interpolating polynomial
    d = len(tau_root)-1

    # Size of the finite elements
    h = tf/nk

    # Coefficients of the collocation equation
    C = np.zeros((d+1,d+1))

    # Coefficients of the continuity equation
    D = np.zeros(d+1)

    # Dimensionless time inside one control interval
    tau = SX.sym("tau")

    # All collocation time points
    T = np.zeros((nk,d+1))
    for k in range(nk):
        for j in range(d+1):
            T[k,j] = h*(k + tau_root[j])

    # For all collocation points
    for j in range(d+1):
    # Construct Lagrange polynomials to get the polynomial basis at the collocation point
        L = 1
        for r in range(d+1):
            if r != j:
                L *= (tau-tau_root[r])/(tau_root[j]-tau_root[r])

        # Evaluate the polynomial at the final time to get the coefficients of the continuity equation
        lfcn = Function('lfcn', [tau],[L])
        D[j] = lfcn(1.0)

        # Evaluate the time derivative of the polynomial at all collocation points to get the coefficients of the continuity equation
        tfcn = Function('tfcn', [tau],[tangent(L,tau)])
        for r in range(d+1):
            C[j,r] = tfcn(tau_root[r])

    # Structure holding NLP variables
    V = struct_symMX([
      (
       entry("X",repeat=[nk+1,d+1],struct=states),
       entry("U",repeat=[nk],shape=nu)
      )
    ])

    vars_lb   = V()
    vars_ub   = V()
    vars_init = V()

    # Set states and its bounds
    vars_init["X",:,:] = repeated(repeated(x_init))
    vars_lb["X",:,:]   = repeated(repeated(x_min))
    vars_ub["X",:,:]   = repeated(repeated(x_max))

    # Set controls and its bounds
    vars_init["U",:] = repeated(u_init)
    vars_lb["U",:]   = repeated(u_min)
    vars_ub["U",:]   = repeated(u_max)

    # State at initial time
    vars_lb["X",0,0] = xi_min
    vars_ub["X",0,0] = xi_max

    # State at end time
    vars_lb["X",-1,0] = xf_min
    vars_ub["X",-1,0] = xf_max

    # Constraint function for the NLP
    g = []
    lbg = []
    ubg = []

    # For all finite elements
    for k in range(nk):

        # For all collocation points
        for j in range(1,d+1):

        # Get an expression for the state derivative at the collocation point
            xp_jk = 0
            for r in range (d+1):
                xp_jk += C[r,j]*V["X",k,r]

            # Add collocation equations to the NLP
            fk = f(T[k][j], V["X",k,j], V["U",k])
            g.append(h*fk - xp_jk)
            lbg.append(np.zeros(nx)) # equality constraints
            ubg.append(np.zeros(nx)) # equality constraints

        # Get an expression for the state at the end of the finite element
        xf_k = 0
        for r in range(d+1):
            xf_k += D[r]*V["X",k,r]

        # Add continuity equation to NLP
        g.append(V["X",k+1,0] - xf_k)
        lbg.append(np.zeros(nx))
        ubg.append(np.zeros(nx))

    # Concatenate constraints
    g = vertcat(*g)

    # Objective function
    f = m(T[nk-1][d],V["X",nk,0],V["U",nk-1])

    # NLP
    nlp = {'x':V, 'f':f, 'g':g}

    ## ----
    ## SOLVE THE NLP
    ## ----

    # Set options
    opts = {}
    opts["expand"] = True
    #opts["ipopt.max_iter"] = 4

    # Allocate an NLP solver
    solver = nlpsol("solver", "ipopt", nlp, opts)
    arg = {}

    # Initial condition
    arg["x0"] = vars_init

    # Bounds on x
    arg["lbx"] = vars_lb
    arg["ubx"] = vars_ub

    # Bounds on g
    arg["lbg"] = np.concatenate(lbg)
    arg["ubg"] = np.concatenate(ubg)

    # Solve the problem
    res = solver(**arg)

    # Print the optimal cost
    print("optimal cost: ", float(res["f"]))

    # Retrieve the solution
    opt = V(res["x"])

    # Get values at the beginning of each finite element
    x0_opt = opt["X",:,0,"x",0]
    x1_opt = opt["X",:,0,"x",1]
    x2_opt = opt["X",:,0,"L"]

    u_opt = opt["U",:,0]

    tgrid = np.linspace(0,tf,nk+1)
    tgrid_u = np.linspace(0,tf,nk)
    
    return(x0_opt,x1_opt,tgrid,u_opt,tgrid_u,float(res["f"]))


# Computations 









In [3]:
results_folder = 'PickleResults/Simulations'
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

### Variable Effort Sensitivity:
- Spatial Penalty: Heaviside Diffuse/Localized 
- Effort: Kinetic energy, Force

In [4]:
kxpenalty_list = [1,100]  # steepness of penalty cost (the higher the steeper: 100 very steep, 1 very diffused)
file_to_save_name_list = [results_folder + '/DiffuseHeavisideSpatialCostEffort.pickle', 
                          results_folder + '/SteepHeavisideSpatialCostEffort.pickle']
ampxpenalty = 10.0 # height of the penality cost
xb = 0.1
xi = 0.1     # initial position
xf = 0.1     # final position 
tf = 7.     # final time (duration of experiment)
vtapis = 0.1

mass = 1
tau_res = 1.8*mass # like humans, or following vector [.9, 1.8, 3.6]
nk = 200 # this is too slow with 500 collocation points

a_list = [.1,1,2,5,10,100] # either [0,.2,.5,1,2] or [5,10,50]
n_comb = len(a_list)
b = 1

In [5]:
for ind_k, kxpenalty in enumerate(kxpenalty_list):
    
    kxpenalty = kxpenalty_list[ind_k]

    # speed 
    xk_vect_speed_heav = np.zeros((nk+1,n_comb))
    xdotk_vect_speed_heav = np.zeros((nk+1,n_comb))
    tk_vect_speed_heav = np.zeros((nk+1,n_comb))
    acck_vect_speed_heav = np.zeros((nk,n_comb))
    tak_vect_speed_heav = np.zeros((nk,n_comb))
    oc_vect_speed_heav = np.zeros((1,n_comb))


    # force
    xk_vect_force_heav = np.zeros((nk+1,n_comb))
    xdotk_vect_force_heav = np.zeros((nk+1,n_comb))
    tk_vect_force_heav = np.zeros((nk+1,n_comb))
    acck_vect_force_heav = np.zeros((nk,n_comb))
    tak_vect_force_heav = np.zeros((nk,n_comb))
    oc_vect_force_heav = np.zeros((1,n_comb))

    for ic in range(n_comb):

        print("ic is " + str(ic))
        a = a_list[ic]
        ###############################################

        xk,xdotk,tk,acck,tak,oc = optimal_trajectory(cost='speed_heaviside', a=a, b=b,
                                                     xb=xb, xi=xi, xf=xf, tf=tf,
                                                     vtapis=vtapis, tau_res=tau_res, mass=mass,
                                                     ampxpenalty=ampxpenalty, kxpenalty=kxpenalty,
                                                     nk=nk)
        xk_vect_speed_heav[:,ic] = xk
        xdotk_vect_speed_heav[:,ic] = xdotk
        tk_vect_speed_heav[:,ic] = tk
        acck_vect_speed_heav[:,ic] = acck
        tak_vect_speed_heav[:,ic] = tak    
        oc_vect_speed_heav[:,ic] = oc     

        ###############################################

        xk,xdotk,tk,acck,tak,oc = optimal_trajectory(cost='force_heaviside', a=a, b=b,
                                                     xb=xb, xi=xi, xf=xf, tf=tf,
                                                     vtapis=vtapis, tau_res=tau_res, mass=mass,
                                                     ampxpenalty=ampxpenalty, kxpenalty=kxpenalty,
                                                     nk=nk)        
        xk_vect_force_heav[:,ic] = xk
        xdotk_vect_force_heav[:,ic] = xdotk
        tk_vect_force_heav[:,ic] = tk
        acck_vect_force_heav[:,ic] = acck
        tak_vect_force_heav[:,ic] = tak    
        oc_vect_force_heav[:,ic] = oc 

    ###############################################
    ###############################################    
    file_to_save_name = file_to_save_name_list[ind_k]
    results = {}
    for i in ('xk_vect_speed_heav','xdotk_vect_speed_heav', 'tk_vect_speed_heav',
              'acck_vect_speed_heav','tak_vect_speed_heav','oc_vect_speed_heav',
              'xk_vect_force_heav','xdotk_vect_force_heav', 'tk_vect_force_heav',
              'acck_vect_force_heav','tak_vect_force_heav','oc_vect_force_heav'):
        results[i] = locals()[i]                    

    with open(file_to_save_name, 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL) 

ic is 0

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    10796
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1200

Total number of variables............................:     2610
                     variables with only lower bounds:        1
                variables with lower and upper bounds:     1806
                     variables with only upper bounds:        0
Total number

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    10796
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1400

Total number of variables............................:     2610
                     variables with only lower bounds:        1
                variables with lower and upper bounds:     1806
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2400
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

ic is 3
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    10796
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1200

Total number of variables............................:     2610
                     variables with only lower bounds:        1
                variables with lower and upper bounds:     1806
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2400
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  l

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    10796
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1400

Total number of variables............................:     2610
                     variables with only lower bounds:        1
                variables with lower and upper bounds:     1806
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2400
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  1.6045418e+01 5.00e-09 3.71e-14  -8.6 3.78e-05  -9.1 1.00e+00 1.00e+00h  1

Number of Iterations....: 20

                                   (scaled)                 (unscaled)
Objective...............:   1.6045417961051584e+01    1.6045417961051584e+01
Dual infeasibility......:   3.7071973773253480e-14    3.7071973773253480e-14
Constraint violation....:   4.9990254019638414e-09    4.9990254019638414e-09
Complementarity.........:   8.7801061980346278e-09    8.7801061980346278e-09
Overall NLP error.......:   8.7801061980346278e-09    8.7801061980346278e-09


Number of objective function evaluations             = 24
Number of objective gradient evaluations             = 21
Number of equality constraint evaluations            = 24
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 21
Number of inequality constraint Jacobian evaluations = 0
Num

...

  19  3.8957171e-01 1.04e-07 2.81e-04  -8.6 7.97e-04 -12.6 1.00e+00 1.25e-01f  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  3.8957187e-01 2.68e-08 6.64e-04  -8.6 8.75e-04 -13.1 6.02e-01 1.00e+00h  1
  21  3.8957199e-01 1.04e-09 8.34e-11  -8.6 1.73e-04 -13.5 1.00e+00 1.00e+00h  1

Number of Iterations....: 21

                                   (scaled)                 (unscaled)
Objective...............:   3.8957198768851009e-01    3.8957198768851009e-01
Dual infeasibility......:   8.3392694847700508e-11    8.3392694847700508e-11
Constraint violation....:   1.0421767430696703e-09    1.0421767430696703e-09
Complementarity.........:   3.1456029670079493e-09    3.1456029670079493e-09
Overall NLP error.......:   3.1456029670079493e-09    3.1456029670079493e-09


Number of objective function evaluations             = 25
Number of objective gradient evaluations             = 22
Number of equality constraint evaluations            = 25
Number of ine

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  3.9681385e-01 3.99e-08 4.98e-04  -8.6 3.62e-04 -13.1 1.00e+00 1.25e-01f  4
  21  3.9681424e-01 1.00e-08 1.85e-03  -8.6 3.78e-04 -13.5 4.25e-01 1.00e+00h  1
  22  3.9681427e-01 8.13e-11 5.09e-12  -8.6 3.41e-05 -14.0 1.00e+00 1.00e+00h  1

Number of Iterations....: 22

                                   (scaled)                 (unscaled)
Objective...............:   3.9681426636841349e-01    3.9681426636841349e-01
Dual infeasibility......:   5.0898249527419061e-12    5.0898249527419061e-12
Constraint violation....:   8.1292619921204763e-11    8.1292619921204763e-11
Complementarity.........:   2.5924096472675086e-09    2.5924096472675086e-09
Overall NLP error.......:   2.5924096472675086e-09    2.5924096472675086e-09


Number of objective function evaluations             = 29
Number of objective gradient evaluations             = 23
Number of equality constraint evaluations            = 29
Number of ine

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  4.0549723e-01 6.87e-08 8.59e-04  -8.6 2.39e-04 -13.1 1.00e+00 3.12e-02f  6
  21  4.0549838e-01 1.32e-08 1.33e-08  -8.6 2.75e-04 -13.5 1.00e+00 1.00e+00h  1
  22  4.0549820e-01 9.98e-10 2.07e-03  -9.0 7.55e-05 -14.0 1.00e+00 1.00e+00h  1
  23  4.0549820e-01 9.53e-11 3.04e-12  -9.0 2.33e-05 -14.5 1.00e+00 1.00e+00h  1

Number of Iterations....: 23

                                   (scaled)                 (unscaled)
Objective...............:   4.0549820015585664e-01    4.0549820015585664e-01
Dual infeasibility......:   3.0394704710968728e-12    3.0394704710968728e-12
Constraint violation....:   9.5346194362786992e-11    9.5346194362786992e-11
Complementarity.........:   9.6647825812816685e-10    9.6647825812816685e-10
Overall NLP error.......:   9.6647825812816685e-10    9.6647825812816685e-10


Number of objective function evaluations             = 38
Number of objective gradient evaluations        

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  4.1103539e-01 2.08e-07 1.18e-02  -8.6 2.57e-04 -13.1 1.00e+00 2.93e-02f  6
  21  4.1103716e-01 3.18e-08 1.76e-08  -8.6 3.01e-04 -13.5 1.00e+00 1.00e+00h  1
  22  4.1103725e-01 3.76e-10 2.37e-11  -8.6 3.28e-05 -14.0 1.00e+00 1.00e+00h  1

Number of Iterations....: 22

                                   (scaled)                 (unscaled)
Objective...............:   4.1103724815580317e-01    4.1103724815580317e-01
Dual infeasibility......:   2.3724038796298149e-11    2.3724038796298149e-11
Constraint violation....:   3.7597362202712592e-10    3.7597362202712592e-10
Complementarity.........:   2.6918647396654491e-09    2.6918647396654491e-09
Overall NLP error.......:   2.6918647396654491e-09    2.6918647396654491e-09


Number of objective function evaluations             = 36
Number of objective gradient evaluations             = 23
Number of equality constraint evaluations            = 36
Number of ine

  19  4.2709396e-01 2.66e-13 2.52e-14  -8.6 2.76e-07 -12.6 1.00e+00 1.00e+00h  1

Number of Iterations....: 19

                                   (scaled)                 (unscaled)
Objective...............:   4.2709395709310843e-01    4.2709395709310843e-01
Dual infeasibility......:   2.5202062658991053e-14    2.5202062658991053e-14
Constraint violation....:   2.6569024758060777e-13    2.6569024758060777e-13
Complementarity.........:   2.5622626961694567e-09    2.5622626961694567e-09
Overall NLP error.......:   2.5622626961694567e-09    2.5622626961694567e-09


Number of objective function evaluations             = 25
Number of objective gradient evaluations             = 20
Number of equality constraint evaluations            = 25
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 20
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 19
Total CPU secs in IPOPT (w

### Variable Rat's Weight:
- Spatial Penalty: Heaviside Diffuse/Localized 
- Effort: Kinetic energy, Force

In [ ]:
kxpenalty_list = [1,100]  # steepness of penalty cost (the higher the steeper: 100 very steep, 1 very diffused)
file_to_save_name_list = [results_folder + '/DiffuseHeavisideSpatialCostMass.pickle', 
                          results_folder + '/SteepHeavisideSpatialCostMass.pickle']
ampxpenalty = 10.0 # height of the penality cost
xb = 0.1
xi = 0.1     # initial position
xf = 0.1     # final position 
tf = 7.     # final time (duration of experiment)
vtapis = 0.1

nk = 200 # this is too slow with 500 collocation points

a = .1 
b = 1 
mass_list = [1,10,20,50,100,1000]
n_comb = len(mass_list) 
tau_res_list = [1.8*i for i in mass_list] # like humans, or following vector [.9, 1.8, 3.6]


In [ ]:
for ind_k, kxpenalty in enumerate(kxpenalty_list):
    
    kxpenalty = kxpenalty_list[ind_k]

    # speed 
    xk_vect_speed_heav = np.zeros((nk+1,n_comb))
    xdotk_vect_speed_heav = np.zeros((nk+1,n_comb))
    tk_vect_speed_heav = np.zeros((nk+1,n_comb))
    acck_vect_speed_heav = np.zeros((nk,n_comb))
    tak_vect_speed_heav = np.zeros((nk,n_comb))
    oc_vect_speed_heav = np.zeros((1,n_comb))


    # force
    xk_vect_force_heav = np.zeros((nk+1,n_comb))
    xdotk_vect_force_heav = np.zeros((nk+1,n_comb))
    tk_vect_force_heav = np.zeros((nk+1,n_comb))
    acck_vect_force_heav = np.zeros((nk,n_comb))
    tak_vect_force_heav = np.zeros((nk,n_comb))
    oc_vect_force_heav = np.zeros((1,n_comb))

    for ic in range(n_comb):

        print("ic is " + str(ic))
        mass = mass_list[ic]
        tau_res = tau_res_list[ic]
        print(tau_res)
        ###############################################

        xk,xdotk,tk,acck,tak,oc = optimal_trajectory(cost='speed_heaviside', a=a, b=b,
                                                     xb=xb, xi=xi, xf=xf, tf=tf,
                                                     vtapis=vtapis, tau_res=tau_res, mass=mass,
                                                     ampxpenalty=ampxpenalty, kxpenalty=kxpenalty,
                                                     nk=nk)
        xk_vect_speed_heav[:,ic] = xk
        xdotk_vect_speed_heav[:,ic] = xdotk
        tk_vect_speed_heav[:,ic] = tk
        acck_vect_speed_heav[:,ic] = acck
        tak_vect_speed_heav[:,ic] = tak    
        oc_vect_speed_heav[:,ic] = oc     

        ###############################################

        xk,xdotk,tk,acck,tak,oc = optimal_trajectory(cost='force_heaviside', a=a, b=b,
                                                     xb=xb, xi=xi, xf=xf, tf=tf,
                                                     vtapis=vtapis, tau_res=tau_res, mass=mass,
                                                     ampxpenalty=ampxpenalty, kxpenalty=kxpenalty,
                                                     nk=nk)        
        xk_vect_force_heav[:,ic] = xk
        xdotk_vect_force_heav[:,ic] = xdotk
        tk_vect_force_heav[:,ic] = tk
        acck_vect_force_heav[:,ic] = acck
        tak_vect_force_heav[:,ic] = tak    
        oc_vect_force_heav[:,ic] = oc 

    ###############################################
    ###############################################    
    file_to_save_name = file_to_save_name_list[ind_k]
    results = {}
    for i in ('xk_vect_speed_heav','xdotk_vect_speed_heav', 'tk_vect_speed_heav',
              'acck_vect_speed_heav','tak_vect_speed_heav','oc_vect_speed_heav',
              'xk_vect_force_heav','xdotk_vect_force_heav', 'tk_vect_force_heav',
              'acck_vect_force_heav','tak_vect_force_heav','oc_vect_force_heav'):
        results[i] = locals()[i]                    

    with open(file_to_save_name, 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL) 